In [1]:
!pip install praat-parselmouth

In [2]:
from pandas import DataFrame

data = DataFrame(columns=['person', 'gender', 'length(s)', 'jitter(%)', 'shimmer(%)'])
print(data)

Empty DataFrame
Columns: [person, gender, length(s), jitter(%), shimmer(%)]
Index: []


In [3]:
import json
import wave

def parsing_json(path, title):
  with open(path+"\\"+title, "r", encoding="UTF-8") as f:
    data = json.load(f)
    gender = data['dataSet']['typeInfo']['speakers'][0]['gender']
    return gender

def getDurationSec(wav_file):
  audio = wave.open(wav_file)
  frames = audio.getnframes()
  rate = audio.getframerate()
  duration = frames/float(rate)
  return duration

In [4]:
import parselmouth
from parselmouth.praat import call

def measure_pitch(f0min, f0max, unit, wav_file):
  voiceID = parselmouth.Sound(wav_file)
  sound = parselmouth.Sound(voiceID)
  pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
  localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
  localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

  return localJitter, localShimmer

In [5]:
import os

root_path_dir = 'C:\\Users\\user\\Desktop\\data\\code\\wav'

file_list = os.listdir(root_path_dir) 
person_count=0

for folder1 in file_list: # 'D01', 'D02'
    folder1_list = os.listdir(root_path_dir+'\\'+folder1) 
    print(">>>>", folder1)
    
    for folder2 in folder1_list: # E01, E02 ...
        folder2_list = os.listdir(root_path_dir+'\\'+folder1+'\\'+folder2) 
        print(">>>>>>>>", folder2)
        
        for folder3 in folder2_list: # S000001, S000002 ...
            print(">>>>>>>>>>>>", folder3)
            person_count += 1
            
            # 1) S000001 폴더 안의 JSON 파일 파싱하기
            json_path = root_path_dir+'\\'+folder1+'\\'+folder2+'\\'+folder3
            json_title = folder3+'.json'
            gender = parsing_json(json_path, json_title)
            
            wav_file_list = os.listdir(json_path)
            
            sum_duration = 0
            
            for wav_file_title in wav_file_list:
                if not wav_file_title.endswith('.wav'):
                    continue
                new_data = {'gender': gender, 'person': person_count}
                wav_file = json_path+'\\'+wav_file_title
                sum_duration += getDurationSec(wav_file)
                
                # 2) 음성 파일 변화율 계산하기
                if gender == '남':
                    jitter, shimmer = measure_pitch(85, 196, "Hertz", wav_file)
                    new_data['jitter(%)'] = jitter*100
                    new_data['shimmer(%)'] = shimmer*100
                else:
                    jitter, shimmer = measure_pitch(155, 334, "Hertz", wav_file)
                    new_data['jitter(%)'] = jitter*100
                    new_data['shimmer(%)'] = shimmer*100
                
                new_data['length(s)'] = sum_duration
                data = data.append(new_data, ignore_index=True)

>>>> D01
>>>>>>>> E01
>>>>>>>>>>>> S000001
>>>>>>>>>>>> S000002
>>>>>>>>>>>> S000003
>>>>>>>>>>>> S000004
>>>>>>>>>>>> S000005
>>>>>>>>>>>> S000006
>>>>>>>>>>>> S000007
>>>>>>>>>>>> S000008
>>>>>>>>>>>> S000009
>>>>>>>>>>>> S000010
>>>>>>>>>>>> S000011
>>>>>>>>>>>> S000012
>>>>>>>>>>>> S000013
>>>>>>>>>>>> S000014
>>>>>>>>>>>> S000015
>>>>>>>>>>>> S000016
>>>>>>>>>>>> S000017
>>>>>>>>>>>> S000018
>>>>>>>>>>>> S000019
>>>>>>>>>>>> S000020
>>>>>>>>>>>> S000021
>>>>>>>>>>>> S000022
>>>>>>>>>>>> S000023
>>>>>>>>>>>> S000024
>>>>>>>>>>>> S000025
>>>>>>>>>>>> S000026
>>>>>>>>>>>> S000027
>>>>>>>>>>>> S000029
>>>>>>>>>>>> S000030
>>>>>>>>>>>> S000031
>>>>>>>>>>>> S000032
>>>>>>>>>>>> S000033
>>>>>>>>>>>> S000034
>>>>>>>>>>>> S000035
>>>>>>>>>>>> S000036
>>>>>>>>>>>> S000037
>>>>>>>>>>>> S000038
>>>>>>>>>>>> S000039
>>>>>>>>>>>> S000040
>>>>>>>>>>>> S000041
>>>>>>>>>>>> S000042
>>>>>>>>>>>> S000043
>>>>>>>>>>>> S000044
>>>>>>>>>>>> S000045
>>>>>>>>>>>> S000046
>>>>>>>>>>>> S000047
>>>>>>>>>>>>

In [6]:
data.to_csv('C:\\Users\\user\\Desktop\\data\\code\\tone_data.csv', header=False, index=False)